Habilidades que Necesitarás

Para sobrevivir en Runway Tech, necesitarás:

    Diseñar una arquitectura de red neuronal con la precisión de una pasarela de alta costura.
    Dominar la propagación hacia adelante y hacia atrás (forward propagation y backpropagation) con la elegancia de una colección de otoño/invierno.
    Domar funciones de activación como ReLU y Softmax, haciéndolas desfilar sin tropiezos.
    Perfeccionar la función de pérdida, porque cada pixel fuera de lugar es una catástrofe.
    Dominar el descenso de gradiente, para que los pesos de tu modelo no sigan tomando malas decisiones (como ese vestido de la temporada pasada).
    Combatir el sobreajuste con la precisión de un sastre de élite — Dropout es tu aliado.


In [1]:
import numpy as np